## Setup libs

In [ ]:
import importlib
import brikasutils as bu
importlib.reload(bu)
import shared_utils as utils
from shared_utils import systemMsg, userMsg, assistantMsg
importlib.reload(utils)
import survey
importlib.reload(survey)
import persona
importlib.reload(persona)

import ollama
import numpy as np
from numpy.linalg import norm
import pandas as pd
import os
import re
import time
import json
from typing import List
from openai import OpenAI

# **Analysis**

### THE REAL DEAL

In [ ]:
# ################### Retrieval below ##################
surv = {}
surv['POSSIBLE_ANSWERS'] = ["I LIKE IT", "I EXPECT IT", "I AM NEUTRAL", "I CAN TOLERATE IT", "I DISLIKE IT", "SOMEWHAT DISAGREE", "DISAGREE", "NEUTRAL", "SOMEWHAT AGREE", "AGREE"]

def extract_possible_answer(value):
    for phrase in surv['POSSIBLE_ANSWERS']:
        pattern = r'(?i)' + re.escape(phrase)
        match = re.search(pattern, value)
        if match:
            return match.group()
    return value  # Return the original value if no possible answer is found

############ Invalid Answers ##################
def get_invalid_answers(value):
    if pd.isna(value):
        return ""
    elif value == "NaN":
        return ""
    elif value in surv['POSSIBLE_ANSWERS']:
        return ""
    else:
        return value
    
# surv = survey.PersonalitySurvey()
# surv = survey.KanoSurvey()

In [ ]:
# Load the simulation results
# both personality and game WIP

dfs = []
filenames = []
# changed_values_per_column = {}

# List of folder paths
# folder_paths = ['simulations/local/video-game/', 'simulations/local/personality/']
# folder_paths = ['batch/output/_pers']
folder_paths = ['batch/output/llama3_7b/pers']#, 'batch/output/_pers']
# pers_folder_path = ['batch/output/_pers']
# kano_folder_path = ['batch/output/_kano']

# Read the first CSV file from the first folder to get the 'question' column
first_folder_path = folder_paths[0]
first_file_path = os.path.join(first_folder_path, os.listdir(first_folder_path)[0])
first_df = pd.read_csv(first_file_path)
questions = first_df['question']
# csv_loop_counter = 0

for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            # Store the original 'answer' column
            # original_answers = df['answer'].copy()
            
            # # remove all characters from a black list from the column answer
            df['answer'] = df['answer'].apply(lambda x: x.strip())
            for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
                df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))
            df['answer'] = df['answer'].str.upper()
            df['answer'] = df['answer'].apply(extract_possible_answer)
            # df['answer'] = df['answer'].apply(get_invalid_answers)
            
            # changed_format = (df['answer'] != original_answers).sum()
            # same_format = (df['answer'] == original_answers).sum()
            
            # Calculate the number of changed values for each column
            # for col in df.columns:
            #     if col not in changed_values_per_column:
            #         changed_values_per_column[col] = 0
                # changed_values_per_column[col] += (df[col] != original_answers[col]).sum()

            filename_without_ext = filename[:-4]  # Remove the '.csv' extension
            filename_without_simulation = (
                filename_without_ext.replace('_simulation', '')
                                    # .replace('video-', '')
            )  # Remove '_simulation' from the filename
            filenames.append(filename_without_simulation)
            dfs.append(df['answer'])
            # csv_loop_counter += 1

answers_df = pd.concat(dfs, axis=1, keys=filenames)
# answers_df.insert(0, 'question', questions)  # Insert the 'question' column at the beginning
#print(f"Values changed: {changed_format}")
df = answers_df
df[:6]

#### SURVEY (both PERSONALITY and KANO)

In [ ]:
surv = survey.PersonalitySurvey()
# df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)
# df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: remap_dict.get(x, x))
df['isValid'] = df.iloc[:, 1:].map(lambda x: x in surv.POSSIBLE_ANSWERS).all(axis=1)
if df['isValid'].all():
    df = df.drop('isValid', axis=1)
    print("All answers were valid")
else:
    print("Some answers were not valid")


In [ ]:
df.head()

In [ ]:
surv = survey.PersonalitySurvey()
# surv = survey.KanoSurvey()

# # Add airidas and elias answers
# air = surv.test_answers["airidas"]
# answers_df.insert(1, "airidas", air[:len(df)])
eli = surv.test_answers["elias"]
df.insert(0, "elias", eli[:len(df)])

# df['answer'] = df['answer'].str.upper()
# df['elias'] = df['elias'].str.upper()
# df = df.dropna()
# df[:5]

### Process data

##### Str2Int

In [ ]:
### Remaps - UNIVERSAL
if isinstance(surv, survey.KanoSurvey):
    remap_dict = {"I EXPECT IT": 5, "I LIKE IT": 4, "I AM NEUTRAL": 3, "I CAN TOLERATE IT": 2, "I DISLIKE IT": 1}
    df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: remap_dict.get(x, x))
    # df['answer'] = df['answer'].map(remap_dict)
    # df['airidas'] = df['airidas'].map(remap_dict)
    # df['elias'] = df['elias'].map(remap_dict)
elif isinstance(surv, survey.PersonalitySurvey):
    remap_dict = {"AGREE": 5, "SOMEWHAT AGREE": 4, "NEUTRAL": 3, "SOMEWHAT DISAGREE": 2, "DISAGREE": 1}
    df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: remap_dict.get(x, x))
    # df.iloc[:, 2:] = df.iloc[:, 2:].map(lambda x: remap_dict[x])
    # remap_cols = df.columns[df.columns.str.contains('|'.join(remap_dict.keys()))]    
    # df['answer'] = df['answer'].map(remap_dict)

df[:2]

In [ ]:
# df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: int(x) if not pd.isna(x) else x)
# df.iloc[:, 1:] = df.iloc[:, 1:].astype(pd.Int64Dtype())

In [ ]:
import pandas as pd
# df.count_non_matching_fields(df, _PersonalitySurvey_)

def count_non_matching_fields(df, surv):
    if isinstance(surv, survey._KanoSurvey_):
        remap_dict = {"I EXPECT IT": 5, "I LIKE IT": 4, "I AM NEUTRAL": 3, "I CAN TOLERATE IT": 2, "I DISLIKE IT": 1}
        possible_answers = list(remap_dict.keys())

        non_matching_counts = (
            df.iloc[:, 1:]
            .applymap(lambda x: x not in possible_answers)
            .sum()
            .rename("non_matching_count")
        )

        df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: remap_dict.get(x, x))

    elif isinstance(surv, survey._PersonalitySurvey_):
        remap_dict = {"AGREE": 5, "SOMEWHAT AGREE": 4, "NEUTRAL": 3, "SOMEWHAT DISAGREE": 2, "DISAGREE": 1}
        possible_answers = list(remap_dict.keys())

        non_matching_counts = (
            df.iloc[:, 2:]
            .applymap(lambda x: x not in possible_answers)
            .sum()
            .rename("non_matching_count")
        )

        df.iloc[:, 2:] = df.iloc[:, 2:].applymap(lambda x: remap_dict.get(x, x))

    return non_matching_counts


In [ ]:
df

In [ ]:
# Assuming your DataFrame is named 'df'
for col in df.columns[1:]:  # Iterate over all columns except the target column
    matches = df.loc[df[col] == df['elias'], col]  # Find rows where values match the target
    print(f"Exact matches for column '{col}': {len(matches)}")

# Compute correlation between each column and the target column
correlations = df.corr()['elias'].drop('elias')
print(correlations)

for col in df.columns[1:]:
    matches = df.loc[df[col] == df['elias'], col]
    print(f"Exact matches for column '{col}': {len(matches)}")

correlations = df.corr()['elias'].drop('elias')
print("\nCorrelations:")
print(correlations)

In [ ]:
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the correlation matrix
corr_matrix = df.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="YlGnBu")
plt.title("Correlation Matrix")
plt.show()

# Assuming 'col1' and 'col2' have the highest positive and negative correlations, respectively
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(df['eli_pers_static-IMPERSONATE_5'], df['elias'])
plt.xlabel('eli_pers_static-IMPERSONATE_5')
plt.ylabel('elias')
plt.title('Scatter Plot: eli_pers_static-IMPERSONATE_5 vs. elias')

plt.subplot(1, 2, 2)
plt.scatter(df['eli_pers_static-ARE_1'], df['elias'])
plt.xlabel('eli_pers_static-ARE_1')
plt.ylabel('elias')
plt.title('Scatter Plot: eli_pers_static-ARE_1 vs. elias')

plt.tight_layout()
plt.show()

In [ ]:
df.iloc[:, 1:] = df.iloc[:, 1:].applymap(lambda x: remap_dict.get(x, x))

In [ ]:
valid_answers = ["I LIKE IT", "I EXPECT IT", "I AM NEUTRAL", "I CAN TOLERATE IT", "I DISLIKE IT", "SOMEWHAT DISAGREE", "DISAGREE", "NEUTRAL", "SOMEWHAT AGREE", "AGREE"]#surv['POSSIBLE_ANSWERS']
# Assuming you have a DataFrame 'df' with columns ['col1', 'col2', 'elias', 'col3', 'col4', ...]
# and a set 'valid_answers' containing the valid answer values
# Get the column names except the first two  and 'elias'
cols = [col for col in df.columns[0:] if col != 'elias']
# Create a dictionary to store the computed values
results = {}
# Function to compute "Exact Matches"
def exact_matches(col):
    # Filter the rows where both columns have valid answers
    valid_rows = (df[col].isin(valid_answers)) & (df['elias'].isin(valid_answers))
    # Compute exact matches only for valid rows
    return (df.loc[valid_rows, col] == df.loc[valid_rows, 'elias']).mean()
# Function to compute "Correlation"
def correlation(col):
    # Filter the rows where both columns have valid answers
    valid_rows = (df[col].isin(valid_answers)) & (df['elias'].isin(valid_answers))
    # Compute correlation only for valid rows
    return df.loc[valid_rows, col].corr(df.loc[valid_rows, 'elias'])
# Compute the "Exact Matches" and "Correlation" for each column against 'elias'
for col in cols:
    key1 = f"Exact Matches - {col} vs elias"
    key2 = f"Correlation - {col} vs elias"
    results[key1] = exact_matches(col)
    results[key2] = correlation(col)
# Sort the columns by their correlation with 'elias' in descending order
sorted_cols = sorted(cols, key=lambda col: results[f"Correlation - {col} vs elias"], reverse=True)
# Print the results sorted by correlation, from highest to lowest
for col in sorted_cols:
    em_key = f"Exact Matches - {col} vs elias"
    corr_key = f"Correlation - {col} vs elias"
    print(f"{col}: Exact Matches = {results[em_key]}, Correlation = {results[corr_key]}")

##### Embed technique

In [ ]:
############ Embed technique ################
# RETRIEVAL_PROMPT = surv['POSSIBLE_ANSWERS']
PRE_DEF_ANSWERS = ["I LIKE IT", "I EXPECT IT", "I AM NEUTRAL", "I CAN TOLERATE IT", "I DISLIKE IT", "SOMEWHAT DISAGREE", "DISAGREE", "NEUTRAL", "SOMEWHAT AGREE", "AGREE"]

embeddings = [ollama.embeddings(model=EMBED_MODEL, prompt=answer)["embedding"] for answer in PRE_DEF_ANSWERS]

FINAL_STRINGS_2_CLEAN = df['answer'] #change to answers_df['answer'] for all simulations
# utils.find_most_similar(ollama.embeddings(model=EMBED_MODEL, prompt=FINAL_STRINGS_2_CLEAN[0])["embedding"], embeddings)
mapped_results = [utils.find_most_similar(string) for string in FINAL_STRINGS_2_CLEAN]
print(mapped_results)
# Generate embeddings for each chunk
embeddings = []

progress, chunks_len = 0, len(chunks) # for progress bar
for chunk_text in chunks:
    progress += 1
    print(f"\rChunk {progress}/{chunks_len}", end="")

    embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
    embeddings.append(embedding)

bu.if_dir_not_exist_make("data/3_embeddings")
bu.quickJSON(AUTO_INFO, f"data/3_embeddings/POSSIBLE_ANSWERS_info.json")
bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"data/3_embeddings/POSSIBLE_ANSWERS_embeddings.json")

##### Trash?

In [ ]:
# Import simulation and info sheet
SIMULATION_NAMEID = "airidas-personality_mixtral_cv1" #f"{SIM_ID}"
df = pd.read_csv(f'simulations/local/personality/{SIMULATION_NAMEID}_simulation.csv')
with open(f'simulations/local/personality/{SIMULATION_NAMEID}_info.json', 'r') as f:
    loaded = json.load(f)
try:
    AUTO_INFO = loaded["info"]
    SETTINGS = loaded["settings"]
    print("Settings and info loaded:")
    for k, v in AUTO_INFO.items():
        print(f"{k}: {v}")
    for k, v in SETTINGS.items():
        print(f"{k}: {v}")
except:
    print("No settings and/or info found")
try:
    if str(type(surv)) != AUTO_INFO["survey_type"]:
        print(f"WARNING: surv variable is not of the same type. {str(type(surv))} != {AUTO_INFO['survey_type']}")
except:
    pass
df.head(n=2)

In [ ]:
# LiveCSV
bu.if_dir_not_exist_make("data/6_sim-clean/results")
res = bu.LiveCSV(f"data/6_sim-clean/elias_runs.csv")

# survey = survey.PersonalitySurvey()
non_matching_counts = count_non_matching_fields(df, surv)
print(non_matching_counts)

In [ ]:
# compute one number of how the percentage of correct answers
result_data = {
    # "Exact Matches - airi": (df['answer'] == df['airidas']).sum() / len(df),
    # "Correlation - airi": df['answer'].corr(df['airidas']),
    "Exact Matches - eli": (df['answer'] == df['elias']).sum() / len(df),
    "Correlation - eli": df['answer'].corr(df['elias']),
}
# compute one number of how the percentage of correct answers
# print(f"Exact Matches: {(df['CLONE_eli'] == df['IRL_eli']).sum() / len(df)}")
# print(f"Correlation: {df['CLONE_eli'].corr(df['IRL_eli'])}")
# df['elias_correct'] = df['CLONE_eli'] == df['IRL_eli']
df.iloc[:, 2:] = df.iloc[:, 2:].applymap(lambda x: remap_dict.get(x, x))
for k, v in result_data.items():
    print(f"{k}: {v}")


In [ ]:
new_res = {
    # "label": None,
    "SIMULATION_NAMEID": SIMULATION_NAMEID,#SIM_ID,
    "timestamp": bu.get_timestamp(),
    "survey_type": str(type(surv)),
    # "temperature": SETTINGS["temperature"],
    # "note": "",
    "exact_matches": result_data["Exact Matches"],
    "corr": result_data["Correlation"],
    "exact_matches_elias": result_data["Exact Matches - elias"],
    "corr_elias": result_data["Correlation - elias"],
}

tmp = bu.convert_dicts_to_table([new_res])
res.append_data(tmp[1], tmp[0])

##### Cleaning

In [ ]:
# def _get_invalid_answers_(value, surv, invalid_answers_df=None):
#   --||---
#     else:
#         if invalid_answers_df is None:
#             invalid_answers_df = pd.DataFrame({'invalid_answer': [value]})
#         elif invalid_answers_df is not None:
#             invalid_answers_df = invalid_answers_df.append({'invalid_answer': value}, ignore_index=True)
#         else:
#             new_row = pd.DataFrame({'invalid_answer': [value]})
#             invalid_answers_df = pd.concat([invalid_answers_df, new_row], ignore_index=True)
#         return value

#################### ^Retrieval ^ ####################
################ VALIDATION #############
for value in values:
    result = _get_invalid_answers_(value, surv, invalid_answers_df)
    if result == "":
        continue
    elif isinstance(result, pd.DataFrame):
        invalid_answers_df = result
    else:
        print(result)

invalid_answers_df = pd.DataFrame.append(invalid_answers_df, pd.DataFrame({'invalid_answer': [value]}), ignore_index=True)
invalid_answers_df = pd.DataFrame(columns=['invalid_answer'])
# Update isValid
df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)

# ################ VALIDATION #############

In [ ]:
# df['airidas'] = df['airidas'].str.upper()
# df['elias'] = df['elias'].str.upper()
# df['answer'] = df['answer'].map(remap_dict)
# df['airidas'] = df['airidas'].map(remap_dict)
# df['elias'] = df['elias'].map(remap_dict)

########################
# df = df.drop(columns=['uppercase_text'])
# df['CLONE_eli'] = df['answer'].apply(extract_uppercase_text)
# df['CLONE_eli'] = df['CLONE_eli'].str.upper()
# .str.upper() or .lower()
# df['answer'] = df['answer'].map(remap_dict, na_action='ignore')
#df['CLONE_eli'] = df['CLONE_eli'].fillna(0).astype(int)